In [1]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name])
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)


for category in starting_category:
    dfs(category)



print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

3231


In [3]:
import json
import urllib.request
from collections import OrderedDict
from tqdm import notebook
from joblib import Parallel, delayed
from time import sleep
from config import *
from infsell import *



def extract_keywords(category,client_auth):
    
    category_id = category[0]
    category_name = category[1]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    while True:	
        try: 
            response_data = requests.post(url_data, json=PAYLOAD)
            break
        except Exception as e:
            print(e)
            pass
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords = get_keywords_database(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in keywords_nb:
        data_keywords[keyword] = category_name
            
       
        

#멀티코어 개수
num_core = 12

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth)
data_keywords = {} 

with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
    parallel(delayed(extract_keywords)(category,client_auth)for category in notebook.tqdm(leaf_category))
    

  0%|          | 0/3231 [00:00<?, ?it/s]

In [4]:
print(len(data_keywords))

1054193


In [14]:
print(data_keywords)

{'니트투피스': '코디세트', '블라우스': '코디세트', '원피스': '코디세트', '나시가디건세트': '코디세트', '치마바지': '코디세트'}


In [5]:

path_database = "data_keywords.json"

with open(path_database, 'w',encoding='utf-8') as f:
    f.write(json.dumps(data_keywords, ensure_ascii=False,indent=4))


In [11]:
with open('database5.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


In [13]:
print(len(data))